# Negative Sentiment Analysis

This notebook demonstrates how to use the negative sentiment classifier to identify and analyze negative comments.

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from analysis.sentiment_classifier import NegativeSentimentClassifier

sns.set_style('whitegrid')
%matplotlib inline

## 1. Initialize the Classifier

In [ ]:
# Initialize the negative sentiment classifier
classifier = NegativeSentimentClassifier(config_path='../config.yaml')

## 2. Load Your Comments Data

Replace with your actual data file path, or use example data:

In [ ]:
# Option 1: Load from CSV file
# df = pd.read_csv('../data/raw/comments.csv')

# Option 2: Use example data
df = pd.DataFrame({
    'text': [
        "This is absolutely terrible and I hate it!",
        "Great product, love it!",
        "Worst experience ever, complete waste of money.",
        "Amazing quality and fast shipping!",
        "Disappointed with the quality, not worth the price.",
        "Best purchase I've made this year!",
        "Buggy and crashes constantly. Very frustrating.",
        "Works perfectly, exactly what I needed.",
        "Terrible customer service, no response to emails.",
        "Exceeded my expectations! Highly recommend."
    ]
})

print(f"Loaded {len(df)} comments")
df.head()

## 3. Classify and Filter Negative Comments

In [ ]:
# Filter to get only negative comments
negative_comments = classifier.filter_negative_comments(df)

print(f"\nNegative comments found: {len(negative_comments)}")
negative_comments.head()

## 4. Analyze Negative Comments Statistics

In [ ]:
# Get detailed statistics
stats = classifier.analyze_negative_comments(negative_comments)

print("\n" + "="*60)
print("NEGATIVE COMMENT ANALYSIS")
print("="*60)
print(f"Total negative comments: {stats['total_negative']}")
print(f"Average negative score: {stats['avg_negative_score']:.3f}")
print(f"Median negative score: {stats['median_negative_score']:.3f}")
print(f"\nBreakdown:")
print(f"  Highly negative (≥0.8): {stats['highly_negative_count']}")
print(f"  Moderately negative (0.6-0.8): {stats['moderately_negative_count']}")
print(f"  Mildly negative (<0.6): {stats['mildly_negative_count']}")

## 5. Visualize Negative Sentiment Distribution

In [ ]:
# Plot negative score distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram of negative scores
axes[0].hist(negative_comments['negative_score'], bins=20, color='#e74c3c', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Negative Score', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Distribution of Negative Sentiment Scores', fontsize=14, fontweight='bold')
axes[0].axvline(x=0.8, color='red', linestyle='--', label='Highly Negative (≥0.8)')
axes[0].axvline(x=0.6, color='orange', linestyle='--', label='Moderately Negative (≥0.6)')
axes[0].legend()

# Bar chart of severity categories
categories = ['Highly\nNegative', 'Moderately\nNegative', 'Mildly\nNegative']
counts = [
    stats['highly_negative_count'],
    stats['moderately_negative_count'],
    stats['mildly_negative_count']
]
colors = ['#c0392b', '#e67e22', '#f39c12']

axes[1].bar(categories, counts, color=colors, edgecolor='black', alpha=0.7)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Negative Comment Severity Breakdown', fontsize=14, fontweight='bold')
axes[1].set_ylim(0, max(counts) * 1.2 if counts else 1)

# Add count labels on bars
for i, (cat, count) in enumerate(zip(categories, counts)):
    axes[1].text(i, count + max(counts)*0.02, str(count), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Display Most Negative Comments

In [ ]:
# Show top 10 most negative comments
print("\n" + "="*80)
print("TOP 10 MOST NEGATIVE COMMENTS")
print("="*80)

text_col = 'text' if 'text' in negative_comments.columns else 'comment'

for i, (idx, row) in enumerate(negative_comments.head(10).iterrows(), 1):
    print(f"\n{i}. NEGATIVE SCORE: {row['negative_score']:.3f}")
    print(f"   {row[text_col][:300]}")
    print("-" * 80)

## 7. Test Individual Comments

In [ ]:
# Test a single comment
test_comment = "This product is awful and doesn't work as advertised!"

result = classifier.classify_comment(test_comment)

print(f"Comment: {test_comment}")
print(f"\nClassification:")
print(f"  Label: {result['label']}")
print(f"  Negative Score: {result['negative_score']:.3f}")
print(f"  Positive Score: {result['positive_score']:.3f}")
print(f"  Is Negative: {result['is_negative']}")

## 8. Export Negative Comments

In [ ]:
# Save negative comments to CSV
output_file = '../results/negative_comments.csv'
negative_comments.to_csv(output_file, index=False)
print(f"Negative comments saved to: {output_file}")

# Optionally, save only highly negative comments
highly_negative = negative_comments[negative_comments['negative_score'] >= 0.8]
if len(highly_negative) > 0:
    output_file_high = '../results/highly_negative_comments.csv'
    highly_negative.to_csv(output_file_high, index=False)
    print(f"Highly negative comments saved to: {output_file_high}")